In [1]:
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as pls
import seaborn as sea
import matplotlib.pyplot as plt

# Loads Data

In [2]:
data = pl.read_csv("/kaggle/input/transactions/synthetic_fraud_data.csv")

In [3]:
data.head()

transaction_id,customer_id,card_number,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,city,city_size,card_type,card_present,device,channel,device_fingerprint,ip_address,distance_from_home,high_risk_merchant,transaction_hour,weekend_transaction,velocity_last_hour,is_fraud
str,str,i64,str,str,str,str,f64,str,str,str,str,str,bool,str,str,str,str,i64,bool,i64,bool,str,bool
"""TX_a0ad2a2a""","""CUST_72886""",6646734767813109,"""2024-09-30 00:00:01.034820+00:…","""Restaurant""","""fast_food""","""Taco Bell""",294.87,"""GBP""","""UK""","""Unknown City""","""medium""","""Platinum Credit""",false,"""iOS App""","""mobile""","""e8e6160445c935fd0001501e4cbac8…","""197.153.60.199""",0,false,0,false,"""{'num_transactions': 1197, 'to…",false
"""TX_3599c101""","""CUST_70474""",376800864692727,"""2024-09-30 00:00:01.764464+00:…","""Entertainment""","""gaming""","""Steam""",3368.97,"""BRL""","""Brazil""","""Unknown City""","""medium""","""Platinum Credit""",false,"""Edge""","""web""","""a73043a57091e775af37f252b3a32a…","""208.123.221.203""",1,true,0,false,"""{'num_transactions': 509, 'tot…",true
"""TX_a9461c6d""","""CUST_10715""",5251909460951913,"""2024-09-30 00:00:02.273762+00:…","""Grocery""","""physical""","""Whole Foods""",102582.38,"""JPY""","""Japan""","""Unknown City""","""medium""","""Platinum Credit""",false,"""Firefox""","""web""","""218864e94ceaa41577d216b1497222…","""10.194.159.204""",0,false,0,false,"""{'num_transactions': 332, 'tot…",false
"""TX_7be21fc4""","""CUST_16193""",376079286931183,"""2024-09-30 00:00:02.297466+00:…","""Gas""","""major""","""Exxon""",630.6,"""AUD""","""Australia""","""Unknown City""","""medium""","""Premium Debit""",false,"""iOS App""","""mobile""","""70423fa3a1e74d01203cf93b51b963…","""17.230.177.225""",0,false,0,false,"""{'num_transactions': 764, 'tot…",false
"""TX_150f490b""","""CUST_87572""",6172948052178810,"""2024-09-30 00:00:02.544063+00:…","""Healthcare""","""medical""","""Medical Center""",724949.27,"""NGN""","""Nigeria""","""Unknown City""","""medium""","""Basic Debit""",false,"""Chrome""","""web""","""9880776c7b6038f2af86bd4e18a1b1…","""136.241.219.151""",1,false,0,false,"""{'num_transactions': 218, 'tot…",true


In [4]:
## Inspects for NA count in data
data.null_count().to_pandas().T.rename(columns={0:"NA_count"})

,NA_count
transaction_id,0
customer_id,0
card_number,0
timestamp,0
merchant_category,0
merchant_type,0
merchant,0
amount,0
currency,0
country,0


# Inspecting Numerical Features

In [5]:
data.select(pls.numeric()).head(5)

card_number,amount,distance_from_home,transaction_hour
i64,f64,i64,i64
6646734767813109,294.87,0,0
376800864692727,3368.97,1,0
5251909460951913,102582.38,0,0
376079286931183,630.6,0,0
6172948052178810,724949.27,1,0
